<a href="https://colab.research.google.com/github/ocisse-jems/spark_bigdata/blob/main/1_PySpark_chargement_manipulation_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chargement et Manipulation d'un dataframe avec PySpark**

In [2]:
# install
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

     |████████████████████████████████| 281.4 MB 50 kB/s 
     |████████████████████████████████| 199 kB 54.3 MB/s 


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## **Lecture d'un fichier .csv**

In [ ]:
path = ""
ville = spark.read.format("csv").option("header", "true").load(path)